In [ ]:
import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Model, losses
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, Input, Lambda,Dropout,Conv2DTranspose
import gc

# Load NPZ files

In [ ]:
orig = '../stack arrays/'
def loadStackArray(path):
    data = np.load(path)
    images, labels = data['images'], data['labels']
    return images, labels


def GetFinalData(path_2017,path_2020):
    print("Loading File...")
    images, labels = loadStackArray(path_2017)
    images_20, labels_20 = loadStackArray(path_2020)
    print("Stacking Images...")
    final_images = np.vstack([images, images_20])
    del images_20, images
    print("Stacking Labels...")
    final_labels = np.vstack([labels, labels_20])
    return final_images, final_labels

In [ ]:
train_images, train_labels = GetFinalData(orig+'training_stacked_arrays_2017.npz', orig+'training_stacked_arrays_2020.npz')

In [ ]:
val_images, val_labels = GetFinalData(orig+'validation_stacked_arrays_2017.npz', orig+'validation_stacked_arrays_2020.npz')

In [ ]:
val_labels.shape

In [ ]:
train_labels.shape

In [ ]:
train_labels = train_labels/255.0
val_labels = val_labels/255.0

In [ ]:
with tf.device("/CPU:0"):
    train_images, train_labels = tf.convert_to_tensor(train_images), tf.convert_to_tensor(train_labels)
    val_images, val_labels = tf.convert_to_tensor(val_images), tf.convert_to_tensor(val_labels)

# U-Net

def build_unet(img_width=img_width, img_height=img_height,img_channels=img_channels):

    inputs = tf.keras.layers.Input((img_width, img_height, img_channels)) 
    normalized_inputs = Lambda(lambda x: x / 255.0)(inputs)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer ='he_normal', padding='same')(normalized_inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
    
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c5)
    
    c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p5)
    c6 = tf.keras.layers.Dropout(0.3)(c6)
    c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
    #p6 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c6)
    
    # Expansive path
    u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c5])
    c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
    
    u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c4])
    c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.2)(c8)
    c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
    
    u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c3])
    c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.2)(c9)
    c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
    u10 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c9)
    u10 = tf.keras.layers.concatenate([u10, c2])
    c10 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u10)
    c10 = tf.keras.layers.Dropout(0.1)(c10)
    c10 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c10)
    
    u11 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c10)
    u11 = tf.keras.layers.concatenate([u11, c1], axis=3)
    c11 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u11)
    c11 = tf.keras.layers.Dropout(0.1)(c11)
    c11 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c11)
    
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c11)
    unet_model = Model(inputs=[inputs], outputs=[outputs], name='U-Net')
    return unet_model

# Loss Function

In [ ]:
def focalLoss(alpha=0.25, gamma=2.0):
    def focal_loss(y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
        
        epsilon = K.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        
        alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_weight = alpha_factor * tf.pow((1 - p_t), gamma)
        bce = K.binary_crossentropy(y_true, y_pred)
        
        loss = focal_weight * bce
        return loss  
    return focal_loss

In [ ]:
def diceLoss(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true + y_pred, axis=[1, 2, 3])
    
    smooth = 1e-6  # Adding a small constant to avoid division by zero
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice  


In [ ]:
def build_loss(alpha=0.25, gamma=2.0, focal_weight=0.5, dice_weight=0.5, **kwargs):
    focal_loss = focalLoss(alpha=alpha, gamma=gamma)
    
    def combined(y_true, y_pred):
        focal = focal_loss(y_true, y_pred)
        dice = diceLoss(y_true, y_pred)
        
        combined_loss = focal_weight * focal + dice_weight * dice
        
        return tf.reduce_mean(combined_loss)
    
    return combined


In [ ]:
# Check if TensorFlow recognizes the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# List available devices
print(tf.config.list_physical_devices())

In [ ]:
img_width = 256
img_height = 256
img_channels = 3

# Original Architecture

In [ ]:
def EncoderBlock(x, filters, useMaxPool=True, dropout = 0.1):
    c1 = Conv2D(filters, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    c2 = Conv2D(filters, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    if useMaxPool:
        p = MaxPooling2D((2,2))(c2)
    else:
        p = c2
    if dropout > 0:
        d = Dropout(dropout)(p)
    else:
        d = p
    return d,c2

def DecoderBlock(x,out, filters, dropout = 0.3):
    C_Transpose = Conv2DTranspose(filters,(2, 2), strides=(2, 2), padding='same')(x)
    merge = concatenate([C_Transpose, out], axis=3)
    c = Conv2D(filters,(3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(merge)
    c = Conv2D(filters, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c)
    if dropout > 0:
        d = Dropout(dropout)(c)
    else:
        d = c
    return d

In [ ]:
def UNET(img_width, img_height, img_channels):
    inputs = Input((img_width, img_height, img_channels))
    normalized_inputs = Lambda(lambda x: x / 255.0)(inputs)
    # Contracting Path
    d1,c1 = EncoderBlock(normalized_inputs,64)
    d2,c2 = EncoderBlock(d1,128)
    d3,c3 = EncoderBlock(d2,256, dropout=0.2)
    d4,c4 = EncoderBlock(d3,512, dropout=0.2)
    # Buttleneck
    d5,c5 = EncoderBlock(d4,1024,False, dropout=0.3)
    # Expansive Path
    c6 = DecoderBlock(c5,c4, 512)
    c7 = DecoderBlock(c6,c3,256,dropout=0.2)
    c8 = DecoderBlock(c7,c2,128, dropout=0.2)
    c9 = DecoderBlock(c8,c1,64,dropout=0.1)
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
    unet_model = Model(inputs=[inputs], outputs=[outputs], name='U-Net') 
    return unet_model

# FIND Best GAMMA

In [ ]:
gammas = [2,3,4]
alpha = 0.25
BATCH_SIZE=32

callbacks = [
    EarlyStopping(patience=2, monitor='val_loss'),
    TensorBoard(log_dir='logs'),
    ModelCheckpoint('solar_panels.keras', save_best_only=True, monitor='val_loss')  
]


for gamma in gammas:
    with tf.device('/CPU:0'):
        unet = UNET(img_width, img_height, img_channels)
        unet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-5),
                      loss=build_loss(gamma=gamma, alpha=alpha),
                      metrics=[tf.keras.metrics.Precision(),
                               tf.keras.metrics.Recall(), 
                               keras.metrics.BinaryIoU(target_class_ids=[1], name='IoU')])
        
    with tf.device('/GPU:1'):
        results = unet.fit(
        train_images, train_labels,
        epochs=10,
        batch_size = BATCH_SIZE,
        callbacks=callbacks,
        validation_data=(val_images, val_labels), verbose=1)
    # Save the training history
    alpha_str = str(alpha).replace('.','')
    with open(f'./unet_alpha_{alpha_str}_gamma_{gamma}.pkl', 'wb') as file:
        pickle.dump(results.history, file)
    unet.save(f'./unet_alpha_{alpha_str}_gamma_{gamma}.h5')
    

# After finding best gamma

In [ ]:
# Parameters
gamma = 2 # best gamma
alpha = 0.25 
BATCH_SIZE = 32

# Callbacks
callbacks = [
    EarlyStopping(patience=2, monitor='val_loss'),
    TensorBoard(log_dir='logs'),
    ModelCheckpoint('solar_panels.keras', save_best_only=True, monitor='val_loss')
]

In [ ]:
with tf.device('/CPU:0'):
        unet = UNET(img_width, img_height, img_channels)
        unet.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss = build_loss(gamma=gamma, alpha=alpha),
        metrics = [
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.BinaryIoU(target_class_ids=[1], name='IoU') # IOU of class 1 only.
        ]
    )

In [ ]:
with tf.device('/GPU:1'):
    results = unet.fit(
        train_images, train_labels,
        epochs=50,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        validation_data=(val_images, val_labels),
        verbose=1
    )

In [ ]:
# Save the training history
with open(f'./unet_alpha_025_gamma_2_final.pkl', 'wb') as file:
    pickle.dump(results.history, file)
# Save the final model
unet.save(f'./UNET_alpha_025_gamma_2_final.h5')